# Collision Avoidance - Train Model

Welcome to this host side Jupyter Notebook!  This should look familiar if you ran through the notebooks that run on the robot.  In this notebook we'll train our image classifier to detect two classes
``free`` and ``blocked``, which we'll use for avoiding collisions.  For this, we'll use a popular deep learning library *PyTorch*

### Upload and extract dataset

Before you start, you should upload the ``dataset.zip`` file that you created in the ``data_collection.ipynb`` notebook on the robot.

You should then extract this dataset by calling the command below

## 若需要解壓縮訓練資料集的資料夾，再進行以下程式功能。
### 執行前，請先取消註解符號 # ，再進行執行。
### 解壓縮需要時間，請耐心等候。

In [ ]:
# !unzip -q dataset.zip -y

# 請設定訓練用資料集的資料夾名稱
## 請務必記得名稱文字前後要加引號 --> ' 

In [1]:
dataset_path = 'yifun_0601_a1'

# 請設定訓練完成後的模型檔(pth檔)名稱
## 請注意!
### 設定檔案名稱要記得指定副檔名為 --> .pth
### 設定檔案名稱要記得在名稱字串前後均加上引號 --> '

In [2]:
model_file = 'yifun_0601_a1.pth'

# 設定神經網路模型訓練的回合數
## 建議至少20回合以上

In [3]:
set_epochs = 20

# <font color="#dd0000">請注意!
# <font color="#dd0000">以下所有程式區塊內容，若非專業人士請勿任意修改程式碼。
# <font color="#dd0000">直接執行所有區塊即可。

In [4]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

## 設定功率模式到5W

In [5]:
import os
password = 'jetbot'
command_set_power = "sudo -S nvpmodel -m1"
print(command_set_power)
os.system('echo %s | %s'%(password,command_set_power))

sudo -S nvpmodel -m1


0

## 開啟散熱風扇

In [6]:
import os
password = 'jetbot'
command = "sudo -S sh -c 'echo 255 > /sys/devices/pwm-fan/target_pwm'"
print(command)
os.system('echo %s | %s'%(password,command))

sudo -S sh -c 'echo 255 > /sys/devices/pwm-fan/target_pwm'


0

### Create dataset instance

Now we use the ``ImageFolder`` dataset class available with the ``torchvision.datasets`` package.  We attach transforms from the ``torchvision.transforms`` package to prepare the data for training.  

#You should see a folder named ``dataset`` appear in the file browser.

In [7]:
dataset = datasets.ImageFolder(
    dataset_path,
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

### Split dataset into train and test sets

Next, we split the dataset into *training* and *test* sets.  The test set will be used to verify the accuracy of the model we train.

In [8]:
#train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])
test_percent = 0.5
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### Create data loaders to load data in batches

We'll create two ``DataLoader`` instances, which provide utilities for shuffling data, producing *batches* of images, and loading the samples in parallel with multiple workers.

In [9]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### Define the neural network

Now, we define the neural network we'll be training.  The *torchvision* package provides a collection of pre-trained models that we can use.

In a process called *transfer learning*, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.

Important features that were learned in the original training of the pre-trained model are re-usable for the new task.  We'll use the ``alexnet`` model.

In [10]:
model = models.alexnet(pretrained=True)

The ``alexnet`` model was originally trained for a dataset that had 1000 class labels, but our dataset only has two class labels!  We'll replace
the final layer with a new, untrained layer that has only two outputs.  

In [11]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

Finally, we transfer our model for execution on the GPU

In [12]:
device = torch.device('cuda')
model = model.to(device)

### Train the neural network

Using the code below we will train the neural network for 30 epochs, saving the best performing model after each epoch.

> An epoch is a full run through our data.

In [ ]:
NUM_EPOCHS = set_epochs
BEST_MODEL_PATH = model_file
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('round %d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy
print('Done!')

round 0: 0.870229
round 1: 0.908397
round 2: 0.862595
round 3: 0.916031
round 4: 0.900763
round 5: 0.938931
round 6: 0.916031
round 7: 0.931298
round 8: 0.916031
round 9: 0.908397
round 10: 0.923664
round 11: 0.908397
round 12: 0.931298
round 13: 0.938931
round 14: 0.916031
round 15: 0.885496
round 16: 0.854962


Once that is finished, you should see a file ``best_model.pth`` in the Jupyter Lab file browser.  Select ``Right click`` -> ``Download`` to download the model to your workstation

## 關閉風扇

In [ ]:
import os
password = 'jetbot'
command = "sudo -S sh -c 'echo 0 > /sys/devices/pwm-fan/target_pwm'"

print(command)

os.system('echo %s | %s'%(password,command))